In [139]:
#importing CSV File and Reading it
import pandas as pd
import csv
import pyspark
from pyspark import SparkContext, SparkConf
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.plotly as py
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr, zscore

In [78]:
#Starting the Spark Session
conf=SparkConf().setAppName("CSE545 Project").set("spark.driver.memory", "12g").set("spark.executor.memory", "6g").set("spark.driver.maxResultSize", "6g")
sc=SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=CSE545 Project, master=local[*]) created by __init__ at <ipython-input-6-4f55b32cb50b>:3 

In [79]:
#Loading GHI Dataset
GHI_rdd1 = sc.textFile("002_AppendixD.tsv").map(lambda line: line.split("\t"))

In [80]:
#Deleting records with more than 1 missing values
def count_x(x):
    cnt = 0
    for i in range(0,len(x)):
        if x[i] == " -":
            cnt += 1
    if cnt > 1:
        return False
    return True

In [81]:
GHI_rdd1 = GHI_rdd1.filter(lambda line: len(line)>1).map(lambda x: [w.replace("\"","") for w in x]).filter(lambda x: count_x(x))

In [82]:
#Converting Strings to floats wherever necessary
def conv_x(x):
    if x[0] != "Country":
        x[1] = float(x[1])
        x[2] = float(x[2])
        x[3] = float(x[3])
        x[4] = float(x[4])
    
    return x

In [83]:
#Transforming <5 values into integer values
def rep_5(x):
    a = []
    cnt = 0
    for i in range(0,len(x)):
        if x[i] == "<5":
            x[i] = 4 - cnt
            x[i] = str(x[i])
            cnt += 1
        a.append(x[i])
        
    return a

In [84]:
#Filling in Missing Values based on the future differences
def fill_x(x):
    if x[1] == " -":
        s = float(x[2]) - float(x[3])
        d = float(x[3]) - float(x[4])
        j = (s + d)/2
        x[1] = str(round(j + float(x[2]),1))
        
    return x

In [85]:
GHI_rdd2 = GHI_rdd1.map(lambda x: rep_5(x)).map(lambda x: fill_x(x)).map(lambda x: conv_x(x))
#print(GHI_rdd2.collect())
#Countries whose records are there in the current dataset
GHI_rdd2_unique_val = GHI_rdd2.map(lambda x: x[0]).distinct().collect()

In [86]:
#Loading other values dataset
GHI_rdd3 = sc.textFile("001_AppendixC.tsv").map(lambda line: line.split("\t")).filter(lambda line: len(line)>1).map(lambda x: [w.replace("\"","") for w in x])

In [87]:
#Filtering values which have corresponding record in GHI dataset
GHI_rdd3 = GHI_rdd3.filter(lambda x: x[0] in GHI_rdd2_unique_val)

In [88]:
#Formatting dataset
GHI_rdd3 = GHI_rdd3.map(lambda x: [x[0],[x[1],x[5],x[9],x[13]],[x[2],x[6],x[10],x[14]],[x[3],x[7],x[11],x[15]],[x[4],x[8],x[12],x[16]]])

In [89]:
#CHeking for the number ofmissing values
def cnt(x):
    a = []
    for i in range(0,len(x)):
        c = 0
        for j in range(0,len(x[i])):
            if x[i][j] == "-":
                c += 1
        a.append(c)
    return a

In [90]:
#Removing the records with at least 2 missing values
def mor_2(x):
    for i in range(0,len(x[1])):
        if x[1][i] == 2:
            return x[0]

In [91]:
GHI_rdd3_toRemove = GHI_rdd3.map(lambda x: (x[0],cnt(x[1:5]))).map(lambda x: mor_2(x)).filter(lambda x: x!=None).collect()

In [92]:
#Updating GHI and values datasets for the final filtered results
GHI_rdd4 = GHI_rdd3.filter(lambda x: x[0] not in GHI_rdd3_toRemove)
GHI_rdd2 = GHI_rdd2.filter(lambda x: x[0] not in GHI_rdd3_toRemove)

In [93]:
#Filling in missing values according to the GHI value
def cal_x0(x,ghi_rdd):
    if x[0] != "Country":
        for j in range(0,len(ghi_rdd.value)):
            if x[0] == ghi_rdd.value[j][0]:
                ghis = ghi_rdd.value[j]
        for i in range(1,len(x)):
            ghi = ghis[i]
            if x[i][0] == "-":
                x[i][1] = round(float(x[i][1]),1)
                x[i][2] = round(float(x[i][2]),1)
                x[i][3] = round(float(x[i][3]),1)
                x[i][0] = round(3*(ghi - (x[i][3]/3) - (x[i][2]/6) - (x[i][1]/6)),1)
            else:
                x[i][0] = round(float(x[i][0]),1)
                x[i][1] = round(float(x[i][1]),1)
                x[i][2] = round(float(x[i][2]),1)
                x[i][3] = round(float(x[i][3]),1)
    return x

In [94]:
def conv_listx(x):
    a = [x[0]]
    for i in range(1,5):
        for j in range(0,4):
            a.append(x[i][j])
    return a

In [95]:
#Call to fill in the missing values
GHI_rdd2_broadcast=sc.broadcast(GHI_rdd2.collect())
GHI_rdd4 = GHI_rdd4.map(lambda x: cal_x0(x,GHI_rdd2_broadcast)).map(lambda x: conv_listx(x))

In [127]:
# #Transforming rdd to pandas dataframe for applying ML and Visulaization techniques
# headers = GHI_rdd2.collect()[0]
# GHI_rdd2=GHI_rdd2.filter(lambda x: x[0]!='Country')

# df0 = pd.DataFrame(GHI_rdd2.collect(), columns=headers)

# headers = GHI_rdd4.collect()[0]
# GHI_rdd4=GHI_rdd4.filter(lambda x: x[0]!='Country')

# df1 = pd.DataFrame(GHI_rdd4.collect(), columns=headers)

In [146]:
def calc_beta(betas, X_test, y_test):
    y_pred = np.matmul(X_test, betas)[:,0]
    print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
    print("Pearson Correlation:", pearsonr(y_test, y_pred))

In [147]:
def RidgeRegression(X, penalty_value = 1.0, learning_rate = 0.0001, n_epochs = 100):

    #Dividing into training and test data
    offset = int(int(X.shape[0]) * 0.8)
    X_test, Y_test = X[offset:,:3], X[offset:,3:]
    X_tf, Y_tf = X[:offset,:3], X[:offset,3:]

    # Conversion to tensors
    X_tf = tf.constant(X_tf, dtype=tf.float32, name="GHI_X")
    Y_tf = tf.constant(Y_tf.reshape(-1,1), dtype=tf.float32, name="GHI_Y")
    Xt_tf = tf.transpose(X_tf)
    penalty = tf.constant(1.0, dtype=tf.float32, name="penalty")
    I = tf.constant(np.identity(int(X_tf.shape[1])), dtype=tf.float32, name="I")
    beta = tf.Variable(tf.random_uniform([int(X_tf.shape[1]), 1], -1., 1.), name = "beta")
    y_pred = tf.matmul(X_tf, beta, name="predictions")
    penalizedCost = tf.reduce_sum(tf.square(Y_tf - y_pred)) + penalty * tf.reduce_sum(tf.square(beta))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(penalizedCost)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epochs):
            if epoch %10 == 0: #print debugging output
                print("Epoch", epoch, "; penalizedCost =", penalizedCost.eval())
            sess.run(training_op)
        #done training, get final beta: 
        best_beta = beta.eval()
    print(best_beta)
    calc_beta(best_beta, X_test, Y_test)

In [144]:
#Transforming rdd to tensors for applying Machine Learning Models
X_ghi=GHI_rdd2.map(lambda x: x[1:])
X_ghi=np.array(X_ghi.collect())
print(X_ghi)


RidgeRegression(X_ghi, 1)
RidgeRegression(X_ghi, .1)#adjusting penalty: worse
RidgeRegression(X_ghi, 10)#better
RidgeRegression(X_ghi, 100)#worse
RidgeRegression(X_ghi, 10, .01) #gives error because learning rate is too high
RidgeRegression(X_ghi, 10, .00001, 10000) #lower learning rate, more epochs: finds finer-grained solution

[[ 50.2  52.7  37.9  33.3]
 [ 20.8  21.6  16.5  11.1]
 [ 17.5  15.6  11.3   9.5]
 [ 65.8  57.5  39.7  32.5]
 [  7.    6.6   5.8   5.4]
 [ 23.8  18.4  11.4   7.7]
 [ 36.5  27.5  15.3   9.6]
 [ 53.6  37.6  32.2  26.5]
 [  5.    4.    3.    2. ]
 [ 44.5  37.5  31.7  24.4]
 [ 36.7  30.3  23.9  17.2]
 [ 33.8  33.   30.7  24.4]
 [ 15.9  11.7   5.4   5.4]
 [  7.9   8.2   7.6   5.4]
 [ 47.   47.9  36.4  27.6]
 [ 45.8  43.6  27.1  22.2]
 [ 40.   39.6  29.5  22.1]
 [ 52.2  50.9  47.   50.9]
 [ 62.5  51.9  50.9  43.5]
 [  5.9   4.    3.    2. ]
 [ 25.9  15.8  11.2   7.5]
 [ 14.6  11.3   9.4   8. ]
 [ 39.1  36.   31.6  25.6]
 [  7.5   6.2   5.    5.3]
 [ 32.9  32.6  35.1  26.5]
 [  7.8   6.2   4.    3. ]
 [ 10.5   5.3   4.    3. ]
 [ 60.3  46.7  35.1  31.4]
 [ 23.8  18.4  15.4  11.6]
 [ 22.3  20.5  16.4  14.4]
 [ 20.1  16.4  16.6  14.7]
 [ 19.5  16.2  12.7  11.1]
 [  7.8   6.2   4.    3. ]
 [ 67.8  56.   40.2  32.3]
 [ 11.5   9.8   9.1   8.1]
 [ 24.2  20.7  17.4  13.8]
 [ 35.2  27.5  23.8  23.2]
 

In [149]:
X_param1=GHI_rdd4.map(lambda x: [x[1],x[5],x[9],x[13]])
X_param1=np.array(X_param1.collect())
print(X_param1)



RidgeRegression(X_param1, 1)
RidgeRegression(X_param1, .1)#adjusting penalty: worse
RidgeRegression(X_param1, 10)#better
RidgeRegression(X_param1, 100)#worse
RidgeRegression(X_param1, 10, .01) #gives error because learning rate is too high
RidgeRegression(X_param1, 10, .00001, 10000) #lower learning rate, more epochs: finds finer-grained solution

[[  37.4   46.1   24.6   23. ]
 [   7.7    7.2    9.7    4.9]
 [   9.1   10.7    7.5    4.6]
 [  65.4   50.    23.7   14. ]
 [   4.7    3.5    4.1    3.6]
 [  52.7   23.8    5.2    4.4]
 [  81.7   23.2    2.6    1.2]
 [  36.1   20.8   16.4   15.1]
 [   8.3    2.3    1.6    1.1]
 [  28.    22.6   12.8   10.3]
 [  35.9   33.4   28.1   20.2]
 [  26.8   35.6   31.9   26. ]
 [  14.2   12.     2.6    2.3]
 [   1.9    4.8    5.8    3.4]
 [  22.7   25.4   22.6   20.2]
 [  30.9   29.2   18.7   15.3]
 [  36.5   30.6   14.5    7.9]
 [  47.7   42.6   34.7   58.6]
 [  55.7   40.1   40.7   32.5]
 [   7.4    4.7    4.1    3.7]
 [  23.3   15.9   13.5    9.6]
 [  12.6    9.7    9.5    7.1]
 [  43.7   32.5   33.    28.2]
 [   5.5    5.1    5.1    5.6]
 [  13.3   17.6   16.5   15.4]
 [  20.9   10.4    2.2    1.4]
 [  13.2    3.7    2.3    0.8]
 [  75.7   48.1   24.1   12.8]
 [  29.2   28.1   20.2   13.5]
 [  16.2   18.5   14.6   12.1]
 [   5.8    5.3    4.6    4.5]
 [  16.    11.2   11.6   12.3]
 [  13.5

In [150]:
X_param2=GHI_rdd4.map(lambda x: [x[2],x[6],x[10],x[14]])
X_param2=np.array(X_param2.collect())
print(X_param2)



RidgeRegression(X_param2, 1)
RidgeRegression(X_param2, .1)#adjusting penalty: worse
RidgeRegression(X_param2, 10)#better
RidgeRegression(X_param2, 100)#worse
RidgeRegression(X_param2, 10, .01) #gives error because learning rate is too high
RidgeRegression(X_param2, 10, .00001, 10000) #lower learning rate, more epochs: finds finer-grained solution

[[ 51.3  54.7  51.3  40.9]
 [ 37.2  39.2  23.1  17.9]
 [ 22.9  23.6  13.2  11.7]
 [ 52.9  47.1  29.2  37.6]
 [  7.1   9.9   8.2   7.5]
 [ 23.5  17.7  20.8   9.4]
 [ 30.   24.1  26.8  18. ]
 [ 71.5  54.   43.2  36.1]
 [  7.5   6.    4.2   2.8]
 [ 43.7  39.1  44.7  34. ]
 [ 37.1  33.1  27.2  18.1]
 [ 41.   29.1  31.4  23.5]
 [ 13.3  10.    7.1   7.9]
 [ 12.8  10.2   8.1   6.2]
 [ 40.7  45.5  35.1  27.3]
 [ 58.5  49.2  39.5  32.4]
 [ 36.3  38.2  36.4  31.7]
 [ 42.6  44.6  45.1  42.6]
 [ 47.2  39.3  38.7  39.9]
 [  4.2   3.    2.    1.8]
 [ 38.   17.8   9.8   6.3]
 [ 22.   18.1  12.7  12.2]
 [ 32.2  28.9  28.8  21.2]
 [ 10.7   7.9   5.6   5.2]
 [ 34.2  31.5  39.   29.6]
 [  1.3   1.4   1.3   1. ]
 [  8.6   7.    6.1   4.4]
 [ 34.3  26.5  33.   33.5]
 [ 21.2   8.   10.1   7.1]
 [ 38.1  32.5  26.3  25.2]
 [ 33.1  24.6  30.7  22.3]
 [ 29.5  32.3  20.6  13.6]
 [ 11.4   5.4   4.4   3.8]
 [ 66.9  57.4  46.4  38.4]
 [  4.3   5.8   6.4   4.9]
 [ 30.3  26.3  19.2  17.5]
 [ 33.9  24.1  25.5  25. ]
 

In [151]:
X_param3=GHI_rdd4.map(lambda x: [x[3],x[7],x[11],x[15]])
X_param3=np.array(X_param3.collect())
print(X_param3)


RidgeRegression(X_param3, 1)
RidgeRegression(X_param3, .1)#adjusting penalty: worse
RidgeRegression(X_param3, 10)#better
RidgeRegression(X_param3, 100)#worse
RidgeRegression(X_param3, 10, .01) #gives error because learning rate is too high
RidgeRegression(X_param3, 10, .00001, 10000) #lower learning rate, more epochs: finds finer-grained solution

[[ 11.5  13.4   8.9   9.5]
 [  9.4  12.2   9.4   6.2]
 [  7.1   3.1   4.1   4.1]
 [  7.9   8.7   8.2   4.9]
 [  1.6   1.6   1.2   1.6]
 [  5.    2.5   4.2   4.2]
 [  6.6   9.    6.8   3.1]
 [ 16.1  13.8  17.5  14.3]
 [  2.6   2.    1.9   1.8]
 [ 11.2   9.    8.4   4.5]
 [  3.6   1.6   1.4   1.6]
 [ 13.1   6.    7.2   6.8]
 [  2.9   2.2   1.6   1.9]
 [  3.4   3.2   3.4   2.8]
 [ 15.5  15.7  11.3   7.6]
 [ 13.9  16.9   8.9   9.6]
 [  4.5   6.2   7.3   5.2]
 [  9.7  10.5  12.2   7.2]
 [ 14.6  13.9  15.7  13. ]
 [  0.5   0.5   0.3   0.3]
 [  3.9   2.5   2.6   1.8]
 [  1.7   1.1   0.9   1.1]
 [  7.1   7.1   7.2   8.2]
 [  2.1   1.7   1.    1.4]
 [  8.3   6.9  14.    7.6]
 [  1.6   1.3   1.    1.2]
 [  3.3   2.4   2.4   2. ]
 [ 17.3  19.4  17.   21.5]
 [  2.2   1.5   2.3   2.4]
 [  2.8   3.2   2.7   2.3]
 [  4.3   6.9   7.9   9.5]
 [  1.4   1.5   1.6   2.1]
 [  4.6   2.7   2.5   2.5]
 [  9.2  12.4  10.8   9.9]
 [  9.8   7.2   6.3   5.3]
 [  4.    4.3   3.7   3.4]
 [ 10.8   8.9   8.5  11.1]
 

In [152]:
X_param4=GHI_rdd4.map(lambda x: [x[4],x[8],x[12],x[16]])
X_param4=np.array(X_param4.collect())
print(X_param4)



RidgeRegression(X_param4, 1)
RidgeRegression(X_param4, .1)#adjusting penalty: worse
RidgeRegression(X_param4, 10)#better
RidgeRegression(X_param4, 100)#worse
RidgeRegression(X_param4, 10, .01) #gives error because learning rate is too high
RidgeRegression(X_param4, 10, .00001, 10000) #lower learning rate, more epochs: finds finer-grained solution

[[ 16.8  13.7  11.    9.1]
 [  3.7   2.6   1.8   1.4]
 [  4.5   4.    2.9   2.6]
 [ 22.6  21.7  19.2  15.7]
 [  2.6   2.    1.5   1.3]
 [  4.5   3.    2.    1.4]
 [  9.5   7.4   4.3   3.2]
 [ 13.2   8.8   5.6   3.8]
 [  1.7   1.4   0.7   0.5]
 [ 17.   14.5  11.6  10. ]
 [ 11.5   8.    5.2   3.8]
 [  5.9   8.3   6.2   4.4]
 [  5.5   3.2   1.8   1.6]
 [  2.3   2.1   1.4   1. ]
 [ 20.2  18.6  13.    8.9]
 [ 11.8  10.8   5.2   2.9]
 [ 14.3  15.   11.3   8.8]
 [ 17.6  17.5  15.8  13. ]
 [ 20.9  19.   16.8  13.9]
 [  1.6   1.1   0.9   0.8]
 [  5.2   3.7   1.9   1.1]
 [  3.3   2.5   2.    1.6]
 [  9.7  12.2   7.3   4.5]
 [  1.6   1.3   1.    1. ]
 [ 15.3  14.6  11.7   9.3]
 [  1.1   0.8   0.6   0.4]
 [  1.2   0.8   0.6   0.6]
 [ 11.5  10.1   8.1   6.5]
 [  5.6   4.1   3.5   3.1]
 [  5.2   3.4   2.7   2.2]
 [  7.8   4.7   3.1   2.4]
 [  5.3   3.2   2.2   1.7]
 [  1.9   1.1   0.6   0.3]
 [ 19.5  14.5   8.7   5.9]
 [  2.8   2.5   2.4   2.2]
 [  9.1   8.5   6.9   5.1]
 [ 15.9  11.9   8.7   6.9]
 